In [1]:
import importlib
import keys
importlib.reload(keys)
import json
import pprint
keychain = keys.keychain

pp = pprint.PrettyPrinter(indent=2)

client_id = keychain['youtube']['client_id']
scope = keychain['youtube']['scope']
redirect = keychain['youtube']['redirect_uris'][0]
print(client_id[:20], scope, redirect)

126920892433-e58m7pn ['https://www.googleapis.com/auth/youtube.readonly'] urn:ietf:wg:oauth:2.0:oob


In [2]:
from requests_oauthlib import OAuth2Session

In [3]:
session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)
auth_url, state = session.authorization_url(keychain['youtube']['auth_uri'])
print(auth_url)

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=w59u5XNAfivdgBiMiHVvxxSklEcPAQ


In [4]:
import webbrowser
webbrowser.open(auth_url)

True

In [8]:
code = '4/l4KY1-BA1sZOpV908HPz91m62_FRCFdVXA2V2QZ6XSg'

In [9]:
token = session.fetch_token(keychain['youtube']['token_uri'], 
                                   code=code, 
                                   client_secret=keychain['youtube']['client_secret'])

In [10]:
print(token)

{'access_token': 'ya29.GlsWBcLkqA2sImJ5pRfrkPTOfMKBDYJdXEPmqQKTu7vKOx4LMdfEolBaAF6GeV2rKj-LyVat7zGOP_iIyS9ecHuzrhZd9s0pC2KhZQxRnYsoLGzov0-lBpJWhRQ7', 'expires_in': 3600, 'refresh_token': '1/_nLEtWzgFS1P3dQK9ePq1RCqtRohrYdi2haVnKpSjwk', 'token_type': 'Bearer', 'expires_at': 1512238305.2425623}


In [108]:
importlib.reload(keys)
keychain = keys.keychain
keychain['youtube']

{'auth_provider_x509_cert_url': 'https://www.googleapis.com/oauth2/v1/certs',
 'auth_uri': 'https://accounts.google.com/o/oauth2/auth',
 'client_id': '126920892433-e58m7pn87600743drvlsjsanlcsus2av.apps.googleusercontent.com',
 'client_secret': 'flFfvozgUappY1MIRCPE5Cxc',
 'project_id': 'project5-187402',
 'redirect_uris': ['urn:ietf:wg:oauth:2.0:oob', 'http://localhost'],
 'scope': ['https://www.googleapis.com/auth/youtube.readonly'],
 'token_uri': 'https://accounts.google.com/o/oauth2/token'}

In [123]:
import requests
import pandas as pd
from xml.etree import ElementTree as ET
import io
D = {}
D['access_token'] = token['access_token']
url = 'https://www.googleapis.com/youtube/v3/channels?part=snippet&mine=true'
response = requests.get(url, params=D)
print(response)

<Response [200]>


In [126]:
url = 'https://www.googleapis.com/youtube/v3/videos/?regionCode=US&chart=mostPopular&part=snippet%2CcontentDetails%2Cstatistics&maxResults=10'
popular_vid = requests.get(url, params = D)
popular_vid = popular_vid.json()


In [116]:
popular_vid['items']
data = {'id':[], 'categoryId':[], 'channelId':[], 'channelTitle':[], 'publishedAt':[], 'comment':[]\
        ,'dislike':[],'favorite':[],'like':[],'viewCount':[]}
for item in popular_vid['items']:
    data['id'].append(item['id'])
    data['categoryId'].append(item['snippet']['categoryId'])
    data['channelId'].append(item['snippet']['channelId'])
    data['channelTitle'].append(item['snippet']['channelTitle'])
    data['publishedAt'].append(item['snippet']['publishedAt'])
    data['comment'].append(item['statistics']['commentCount'])
    data['dislike'].append(item['statistics']['dislikeCount'])
    data['favorite'].append(item['statistics']['favoriteCount'])
    data['like'].append(item['statistics']['likeCount'])
    data['viewCount'].append(item['statistics']['viewCount'])
df = pd.DataFrame(data)
df

,categoryId,channelId,channelTitle,comment,dislike,favorite,id,like,publishedAt,viewCount
0,24,UCq0OueAsdxH6b8nyAspwViw,Universal Pictures,6263,2539,0,266EI7hdjFs,27967,2017-12-04T00:41:00.000Z,2137018
1,24,UChi08h4577eFsNXGd3sxYhw,Breakfast Club Power 105.1 FM,2046,433,0,c3BTGJgd8Wo,3929,2017-12-04T13:31:15.000Z,224348
2,24,UC4PooiX37Pld1T8J5SYT-SQ,Good Mythical Morning,1698,532,0,-C5q3oir8gA,13045,2017-12-04T11:00:00.000Z,409304
3,26,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,10994,20584,0,_9YMpuLDnwo,98144,2017-12-03T08:30:00.000Z,10291235
4,17,UCDVYQ4Zhbm3S2dlz7P1GBDg,NFL,6587,591,0,xl3HhY2BTuU,11658,2017-12-04T04:55:56.000Z,1164917


In [125]:
popular_vid


{'etag': '"7991kDR-QPaa9r0pePmDjBEa2h8/IhLh7OltEcRHUcUTsqSTyo97xNI"',
 'items': [{'contentDetails': {'caption': 'false',
    'definition': 'hd',
    'dimension': '2d',
    'duration': 'PT4M47S',
    'licensedContent': True,
    'projection': 'rectangular'},
   'etag': '"7991kDR-QPaa9r0pePmDjBEa2h8/ZeC_67KNpwlTnwVfCp7qb-bQT-Q"',
   'id': 'SxAp27sFaIM',
   'kind': 'youtube#video',
   'snippet': {'categoryId': '22',
    'channelId': 'UCqwxMqUcL-XC3D9-fTP93Mg',
    'channelTitle': 'LIL UZI VERT',
    'description': 'Lil Uzi Vert - The Way Life Goes Remix (Feat. Nicki Minaj)\nDownload/Stream - https://Atlantic.lnk.to/TheWayLifeGoesRemix\n\nWin a chance to celebrate the holidays with Uzi - http://www.liluziofficial.com/win\n\n‘Sampling Oh Wonder’s ‘Landslide’ - http://ohwondermusic.com/’\n\nDirected by: DAPS\nVideo Commissioners: Joseph Boyd & Emmanuelle Cuny\nAssociate Dir., Video Production: Lily Thrall\nCoordinator, Video Content: Austin Gomez\nAssistant, Video Production: Trevor Joseph N

In [121]:
refresh_url = keychain['youtube']['token_uri']
token = session.refresh_token(refresh_url, 
                             client_id=keychain['youtube']['client_id'],
                             client_secret=keychain['youtube']['client_secret'])

In [122]:
print(token)

{'access_token': 'ya29.GlwZBTa9WhKKIDYDVGVAEOOnNWjWZuVeXuje-xk3iQf1TFLDDFemoS2-l5Q_cqO-iFZYib60CExn77pp72v7TExUp6sE0Nr7Zj-BUaC1xfrKarJOfe_LnccWFu6qBQ', 'expires_in': 3600, 'token_type': 'Bearer', 'expires_at': 1512502161.1268606, 'refresh_token': '1/_nLEtWzgFS1P3dQK9ePq1RCqtRohrYdi2haVnKpSjwk'}
